In [16]:
import numpy as np

In [17]:
def load_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]  # Từ là giá trị đầu tiên
            vector = np.array(list(map(float, values[1:])))  # Các giá trị tiếp theo là vector embedding
            embeddings[word] = vector
    return embeddings

# Đọc embedding từ file W2V_150.txt
embeddings_path = '.\dataset\W2V_150.txt'
embeddings = load_embeddings(embeddings_path)

<>:12: SyntaxWarning: invalid escape sequence '\d'
<>:12: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nhuyh\AppData\Local\Temp\ipykernel_13344\3874027511.py:12: SyntaxWarning: invalid escape sequence '\d'
  embeddings_path = '.\dataset\W2V_150.txt'


In [26]:
embeddings['công_nghệ']

array([-0.4684519 , -0.6204796 ,  1.034518  ,  0.9500654 , -0.8292917 ,
        0.1149449 , -0.4122957 ,  0.7012163 , -1.119478  , -1.201047  ,
        0.8173522 ,  0.08329333, -1.416684  , -0.7337989 , -1.273167  ,
        0.8110114 , -0.7506533 ,  1.045421  ,  0.5872381 ,  1.181379  ,
        0.4509734 , -0.1116145 ,  0.6884549 ,  1.025807  ,  1.275964  ,
       -0.9710795 , -0.01143223,  0.2441304 ,  0.2759391 , -0.3112414 ,
        0.6230265 ,  1.08157   ,  1.553475  ,  1.438222  ,  0.04897799,
        0.08947516,  1.634697  , -0.5253603 ,  1.282322  , -0.7748135 ,
       -0.0163918 ,  0.5861691 , -0.1297526 ,  0.8481283 , -0.2779701 ,
        0.4621512 , -0.6370358 , -0.1701398 ,  0.277995  , -0.5310548 ,
        0.7507658 , -0.5637417 , -0.07948318,  0.06691374,  1.843204  ,
       -0.3851503 , -0.00972147,  0.6552926 , -0.6680869 , -0.8480693 ,
        0.5710877 ,  0.4132398 ,  0.00392161,  1.577133  ,  1.100089  ,
        1.971255  , -1.770977  ,  0.6618114 , -1.539468  ,  0.56

In [18]:
def load_visim400(file_path):
    word_pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        next(f)  # Bỏ qua dòng tiêu đề
        for line in f:
            values = line.strip().split('\t')
            word1 = values[0]
            word2 = values[1]
            pos = values[2]
            sim1 = float(values[3])
            sim2 = float(values[4])
            std = float(values[5])
            word_pairs.append((word1, word2, pos, sim1, sim2, std))
    return word_pairs

# Đọc file Visim-400.txt
visim400_path = '.\dataset\Visim-400.txt'
word_pairs = load_visim400(visim400_path)


<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
C:\Users\nhuyh\AppData\Local\Temp\ipykernel_13344\2005201167.py:17: SyntaxWarning: invalid escape sequence '\d'
  visim400_path = '.\dataset\Visim-400.txt'


In [19]:
word_pairs[:5]

[('biến', 'ngập', 'V', 3.13, 5.22, 0.72),
 ('nhà_thi_đấu', 'nhà', 'N', 3.07, 5.12, 1.18),
 ('động', 'tĩnh', 'V', 0.6, 1.0, 0.95),
 ('khuyết', 'ưu', 'N', 0.2, 0.33, 0.4),
 ('cõi_tục', 'cõi_âm', 'N', 0.6, 1.0, 0.95)]

## Công thức Tính cosine similarity

Cosine similarity là một thước đo độ tương đồng giữa hai vector không gian, được tính bằng công thức:

$$
\text{cosine\_similarity}(A, B) = \frac{A \cdot B}{\|A\| \|B\|}
$$

Trong đó:
- \( A \cdot B \) là tích vô hướng của hai vector \( A \) và \( B \).
- \( \|A\| \) và \( \|B\| \) là độ dài (norm) của vector \( A \) và \( B \).

Cosine similarity có giá trị từ -1 đến 1, với 1 biểu thị hai vector hoàn toàn giống nhau, 0 biểu thị hai vector vuông góc (không liên quan), và -1 biểu thị hai vector hoàn toàn đối lập.


In [20]:
# Tính toán độ tương đồng cosine giữa hai vector
def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (abs(norm_vec1) * abs(norm_vec2))

In [25]:
def calculate_similarity_for_word_pairs(word_pairs, embeddings):
    results = []
    for word1, word2, pos, sim1, sim2, std in word_pairs:
        if word1 in embeddings and word2 in embeddings:
            vec1 = embeddings[word1]
            vec2 = embeddings[word2]
            similarity = calculate_cosine_similarity(vec1, vec2)
            results.append((word1, word2, similarity, sim1, sim2, std))
        else:
            results.append((word1, word2, None, sim1, sim2, std))  # Nếu không có embedding cho từ
    return results

# Tính toán độ tương đồng cosine cho các cặp từ
similarities = calculate_similarity_for_word_pairs(word_pairs, embeddings)

# Hiển thị kết quả
for word1, word2, similarity, sim1, sim2, std in similarities:
    if similarity is not None:
        print(f"Độ tương đồng cosine giữa '{word1}' và '{word2}': {similarity:.4f} (sim1={sim1}, sim2={sim2}, std={std})")
    else:
        print(f"Không tìm thấy embedding cho cặp từ: {word1}, {word2} (sim1={sim1}, sim2={sim2}, std={std})")


Độ tương đồng cosine giữa 'biến' và 'ngập': -0.0049 (sim1=3.13, sim2=5.22, std=0.72)
Độ tương đồng cosine giữa 'nhà_thi_đấu' và 'nhà': 0.0825 (sim1=3.07, sim2=5.12, std=1.18)
Độ tương đồng cosine giữa 'động' và 'tĩnh': 0.2771 (sim1=0.6, sim2=1.0, std=0.95)
Độ tương đồng cosine giữa 'khuyết' và 'ưu': 0.1768 (sim1=0.2, sim2=0.33, std=0.4)
Không tìm thấy embedding cho cặp từ: cõi_tục, cõi_âm (sim1=0.6, sim2=1.0, std=0.95)
Độ tương đồng cosine giữa 'thủ_pháp' và 'biện_pháp': 0.4024 (sim1=4.13, sim2=6.88, std=1.26)
Độ tương đồng cosine giữa 'kết_duyên' và 'thành_hôn': 0.4630 (sim1=5.27, sim2=8.78, std=1.06)
Độ tương đồng cosine giữa 'cấp_tiến' và 'bảo_thủ': 0.2569 (sim1=0.87, sim2=1.45, std=1.15)
Độ tương đồng cosine giữa 'nước_lớn' và 'nguy_hiểm': 0.1852 (sim1=1.07, sim2=1.78, std=1.12)
Độ tương đồng cosine giữa 'hoa' và 'nhị': 0.2231 (sim1=2.27, sim2=3.78, std=0.93)
Độ tương đồng cosine giữa 'bất_lợi' và 'thuận_lợi': 0.5349 (sim1=0.33, sim2=0.55, std=0.79)
Độ tương đồng cosine giữa 'phân_

In [20]:
similarities

[('biến', 'ngập', np.float64(-0.004912339469670047), 3.13, 5.22, 0.72),
 ('nhà_thi_đấu', 'nhà', np.float64(0.08252318329211765), 3.07, 5.12, 1.18),
 ('động', 'tĩnh', np.float64(0.27708595986827744), 0.6, 1.0, 0.95),
 ('khuyết', 'ưu', np.float64(0.1767986283562671), 0.2, 0.33, 0.4),
 ('cõi_tục', 'cõi_âm', None, 0.6, 1.0, 0.95),
 ('thủ_pháp', 'biện_pháp', np.float64(0.40236612919430614), 4.13, 6.88, 1.26),
 ('kết_duyên', 'thành_hôn', np.float64(0.46300840201407223), 5.27, 8.78, 1.06),
 ('cấp_tiến', 'bảo_thủ', np.float64(0.25694700889961236), 0.87, 1.45, 1.15),
 ('nước_lớn', 'nguy_hiểm', np.float64(0.18519202240211619), 1.07, 1.78, 1.12),
 ('hoa', 'nhị', np.float64(0.22306960452548538), 2.27, 3.78, 0.93),
 ('bất_lợi', 'thuận_lợi', np.float64(0.5348913029963391), 0.33, 0.55, 0.79),
 ('phân_ly', 'sum_họp', np.float64(0.07770108930396215), 0.47, 0.78, 1.09),
 ('nói_lắp', 'nói_móc', None, 3.53, 5.88, 0.81),
 ('vây_ép', 'phong_toả', None, 3.4, 5.67, 0.95),
 ('diễu_hành', 'tuần_hành', np.float6